# Agent

\begin{align}
p(r_t,u_t,\omega_t,a_t | \omega_{t-1}) \propto \underbrace{p(r_t|u_t) p(u_t|\omega_t)}_{\text{likelihood}} \underbrace{p(\omega_t | \omega_{t-1}, a_t)}_{\text{state transition}}\underbrace{p(a_t)}_{\text{control}} \qquad (1)
\end{align}

\begin{align}
p(r_t,u_t,\omega_t | \omega_{t-1}) \propto p(r_t|\omega_t)p(\omega_t | \omega_{t-1}, u_t)p(u_t) \qquad (2)
\end{align}

\begin{align}
p(r_t,u_t,\omega_t \mid u_{t-1}) \propto \underbrace{p(r_t \mid u_t)}_{\text{response}} \underbrace{ p(u_t \mid u_{t-1}) p(u_t \mid \omega_t)}_{\text{perception }p(u_t \mid u_{t-1},\omega_t)} \underbrace{p(\omega_t)}_{\substack{\text{control} \\ \text{prior}}}  \qquad (3)
\end{align}

In [1]:
using ForneyLab
using ProgressMeter
using LinearAlgebra
using Plots

┌ Info: Precompiling ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9]
└ @ Base loading.jl:1278


In [22]:
# Model (1)
function create_agent_1(horizon)
    # Building the model
    n_samples = horizon
    fg = FactorGraph()

    # State prior
    @RV ω_0 ~ GaussianMeanPrecision(placeholder(:m_ω_0), placeholder(:w_ω_0))

    @RV a ~ GaussianMeanPrecision(placeholder(:m_a), placeholder(:w_a))

    # Transition and observation model
    ω = Vector{Variable}(undef, n_samples)
    u = Vector{Variable}(undef, n_samples)
    r = Vector{Variable}(undef, n_samples)

    ω_i_min = ω_0
    for i in 1:n_samples

        @RV ω[i] = ω_i_min + a

        f(z) = 1/(1+ exp(-z))
        @RV u[i] ~ Nonlinear{Sampling}(ω[i], g=f)
        @RV r[i] ~ GaussianMeanPrecision(u[i], 100.0)

        # Data placeholder
        placeholder(r[i], :r, index=i)

        # Reset state for next step
        ω_i_min = ω[i]
    end
    return fg
end


create_agent_1 (generic function with 1 method)

In [2]:
# Model (2)
function create_agent_2(horizon)
    # Building the model
    n_samples = horizon
    fg = FactorGraph()

    # State prior
    @RV ω_0 ~ GaussianMeanPrecision(placeholder(:m_ω_0), placeholder(:w_ω_0))

    @RV u ~ GaussianMeanPrecision(placeholder(:m_u), placeholder(:w_u))

    # Transition and observation model
    ω = Vector{Variable}(undef, n_samples)
    ωu = Vector{Variable}(undef, n_samples)
    r = Vector{Variable}(undef, n_samples)

    ω_i_min = ω_0
    for i in 1:n_samples

        f(x,z) = 1/(1+ exp(-z*x))
        @RV ω[i] ~ Nonlinear{Sampling}(ω_i_min, u, g=f)

        @RV ωu[i] ~ GaussianMeanPrecision(ω[i], 100.0)

        @RV r[i] ~ Bernoulli(ωu[i])

        # Data placeholder
        placeholder(r[i], :r, index=i)

        # Reset state for next step
        ω_i_min = ωu[i]
    end
    draw()
    q = PosteriorFactorization(ω, ωu, ids=[:Ω :ΩU])
    algo = messagePassingAlgorithm(free_energy=true)
    source_code = algorithmSourceCode(algo, free_energy=true)
end


create_agent_2 (generic function with 1 method)

In [3]:
# Building the model
n_samples = 2
fg = FactorGraph()

# State prior
@RV ω_0 ~ GaussianMeanPrecision(placeholder(:m_ω_0), placeholder(:w_ω_0))

@RV u ~ GaussianMeanPrecision(placeholder(:m_u), placeholder(:w_u))

# Transition and observation model
ω = Vector{Variable}(undef, n_samples)
ωu = Vector{Variable}(undef, n_samples)
r = Vector{Variable}(undef, n_samples)

ω_i_min = ω_0
for i in 1:n_samples

    f(x,z) = 1/(1+ exp(-z*x))
    @RV ω[i] ~ Nonlinear{Sampling}(ω_i_min, u, g=f)

    @RV ωu[i] ~ GaussianMeanPrecision(ω[i], 100.0)

    @RV r[i] ~ Bernoulli(ωu[i])

    # Data placeholder
    placeholder(r[i], :r, index=i)

    # Reset state for next step
    ω_i_min = ωu[i]
end
draw()
q = PosteriorFactorization(ω, ωu, u, ids=[:Ω :ΩU :U])
algo = messagePassingAlgorithm(free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 17456995671846919285 -->
 
 17456995671846919285 
 
 clamp_2 
 
<!-- 137617360431203333 -->
 
 137617360431203333 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 1595157251946666156 -->
 
 1595157251946666156 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 137617360431203333--1595157251946666156 -->
 
 137617360431203333--1595157251946666156 
 
 ω_1 
 1 out 
 2 m 
 
<!-- 10348405999983634769 -->
 
 10348405999983634769 
 
 clamp_1 
 
<!-- 137617360431203333--10348405999983634769 -->
 
 137617360431203333--10348405999983634769 
 
 clamp_1 
 1 out 
 3 w 
 
<!-- 5775962577517957401 -->
 
 5775962577517957401 
 
 g{Sampling} 
 nonlinear_2 
 
<!-- 12728516468098578233 -->
 
 12728516468098578233 
 
 placeholder_r_2 
 
<!-- 638225395594806039 -->
 
 638225395594806039 
 
 Ber 
 bernoulli_2 
 
<!-- 12728516468098578233--638225395594806039 -->
 
 12728516468098578233--638225395594806039 
 
 r_2 
 1 out 
 1 out 
 
<!-- 17910314970211653301 -->
 
 17910314970211653301 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 14497884020957520505 -->
 
 14497884020957520505 
 
 placeholder_w_ω_0 
 
<!-- 17910314970211653301--14497884020957520505 -->
 
 17910314970211653301--14497884020957520505 
 
 w_ω_0 
 1 out 
 3 w 
 
<!-- 9137465564000362079 -->
 
 9137465564000362079 
 
 placeholder_m_ω_0 
 
<!-- 17910314970211653301--9137465564000362079 -->
 
 17910314970211653301--9137465564000362079 
 
 m_ω_0 
 1 out 
 2 m 
 
<!-- 14166576050113124833 -->
 
 14166576050113124833 
 
 𝒩 
 gaussianmeanprecision_4 
 
<!-- 14166576050113124833--17456995671846919285 -->
 
 14166576050113124833--17456995671846919285 
 
 clamp_2 
 1 out 
 3 w 
 
<!-- 14166576050113124833--5775962577517957401 -->
 
 14166576050113124833--5775962577517957401 
 
 ω_2 
 1 out 
 2 m 
 
<!-- 17991384629735124371 -->
 
 17991384629735124371 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 11889332765626330634 -->
 
 11889332765626330634 
 
 placeholder_m_u 
 
<!-- 17991384629735124371--11889332765626330634 -->
 
 17991384629735124371--11889332765626330634 
 
 m_u 
 1 out 
 2 m 
 
<!-- 8160738703519813304 -->
 
 8160738703519813304 
 
 placeholder_w_u 
 
<!-- 17991384629735124371--8160738703519813304 -->
 
 17991384629735124371--8160738703519813304 
 
 w_u 
 1 out 
 3 w 
 
<!-- 1595157251946666156--17910314970211653301 -->
 
 1595157251946666156--17910314970211653301 
 
 ω_0 
 1 out 
 2 in1 
 
<!-- 638225395594806039--14166576050113124833 -->
 
 638225395594806039--14166576050113124833 
 
 ωu_2 
 1 out 
 2 p 
 
<!-- 3368846892928075864 -->
 
 3368846892928075864 
 
 Ber 
 bernoulli_1 
 
<!-- 9719492742121135164 -->
 
 9719492742121135164 
 
 = 
 equ_u_1 
 
<!-- 9719492742121135164--5775962577517957401 -->
 
 9719492742121135164--5775962577517957401 
 
 u 
 3 in2 
 3 3 
 
<!-- 9719492742121135164--17991384629735124371 -->
 
 9719492742121135164--17991384629735124371 
 
 u 
 1 out 
 1 1 
 
<!-- 9719492742121135164--1595157251946666156 -->
 
 9719492742121135164--1595157251946666156 
 
 u 
 3 in2 
 2 2 
 
<!-- 4822233832652181921 -->
 
 4822233832652181921 
 
 = 
 equ_ωu_1_1 
 
<!-- 4822233832652181921--137617360431203333 -->
 
 4822233832652181921--137617360431203333 
 
 ωu_1 
 1 out 
 1 1 
 
<!-- 4822233832652181921--5775962577517957401 -->
 
 4822233832652181921--5775962577517957401 
 
 ωu_1 
 2 in1 
 3 3 
 
<!-- 4822233832652181921--3368846892928075864 -->
 
 4822233832652181921--3368846892928075864 
 
 ωu_1 
 2 p 
 2 2 
 
<!-- 15955734043749553171 -->
 
 15955734043749553171 
 
 placeholder_r_1 
 
<!-- 15955734043749553171--3368846892928075864 -->
 
 15955734043749553171--3368846892928075864 
 
 r_1 
 1 out 
 1 out

LoadError: ArgumentError: The input graph contains a loop around Interface 3 (3) of Equality equ_ωu_1_1
.

In [4]:
draw(q.posterior_factors[:Ω])

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 1595157251946666156 -->
 
 1595157251946666156 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 17910314970211653301 -->
 
 17910314970211653301 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 1595157251946666156--17910314970211653301 -->
 
 1595157251946666156--17910314970211653301 
 
 ω_0 
 1 out 
 2 in1 
 
<!-- 3368846892928075864 -->
 
 3368846892928075864 
 
 Ber 
 bernoulli_1 
 
<!-- 9719492742121135164 -->
 
 9719492742121135164 
 
 = 
 equ_u_1 
 
<!-- 9719492742121135164--1595157251946666156 -->
 
 9719492742121135164--1595157251946666156 
 
 u 
 3 in2 
 2 2 
 
<!-- 5775962577517957401 -->
 
 5775962577517957401 
 
 g{Sampling} 
 nonlinear_2 
 
<!-- 9719492742121135164--5775962577517957401 -->
 
 9719492742121135164--5775962577517957401 
 
 u 
 3 in2 
 3 3 
 
<!-- 17991384629735124371 -->
 
 17991384629735124371 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 9719492742121135164--17991384629735124371 -->
 
 9719492742121135164--17991384629735124371 
 
 u 
 1 out 
 1 1 
 
<!-- 137617360431203333 -->
 
 137617360431203333 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 137617360431203333--1595157251946666156 -->
 
 137617360431203333--1595157251946666156 
 
 ω_1 
 1 out 
 2 m 
 
<!-- 10348405999983634769 -->
 
 10348405999983634769 
 
<!-- 137617360431203333--10348405999983634769 -->
 
 137617360431203333--10348405999983634769 
 
 1 out 
 3 w 
 
<!-- 4822233832652181921 -->
 
 4822233832652181921 
 
 = 
 equ_ωu_1_1 
 
<!-- 4822233832652181921--3368846892928075864 -->
 
 4822233832652181921--3368846892928075864 
 
 ωu_1 
 2 p 
 2 2 
 
<!-- 4822233832652181921--137617360431203333 -->
 
 4822233832652181921--137617360431203333 
 
 ωu_1 
 1 out 
 1 1 
 
<!-- 4822233832652181921--5775962577517957401 -->
 
 4822233832652181921--5775962577517957401 
 
 ωu_1 
 2 in1 
 3 3 
 
<!-- 14497884020957520505 -->
 
 14497884020957520505 
 
<!-- 17910314970211653301--14497884020957520505 -->
 
 17910314970211653301--14497884020957520505 
 
 1 out 
 3 w 
 
<!-- 9137465564000362079 -->
 
 9137465564000362079 
 
<!-- 17910314970211653301--9137465564000362079 -->
 
 17910314970211653301--9137465564000362079 
 
 1 out 
 2 m 
 
<!-- 14166576050113124833 -->
 
 14166576050113124833 
 
 𝒩 
 gaussianmeanprecision_4 
 
<!-- 14166576050113124833--5775962577517957401 -->
 
 14166576050113124833--5775962577517957401 
 
 ω_2 
 1 out 
 2 m 
 
<!-- 17456995671846919285 -->
 
 17456995671846919285 
 
<!-- 14166576050113124833--17456995671846919285 -->
 
 14166576050113124833--17456995671846919285 
 
 1 out 
 3 w 
 
<!-- 11889332765626330634 -->
 
 11889332765626330634 
 
<!-- 17991384629735124371--11889332765626330634 -->
 
 17991384629735124371--11889332765626330634 
 
 1 out 
 2 m 
 
<!-- 8160738703519813304 -->
 
 8160738703519813304 
 
<!-- 17991384629735124371--8160738703519813304 -->
 
 17991384629735124371--8160738703519813304 
 
 1 out 
 3 w 
 
<!-- 638225395594806039 -->
 
 638225395594806039 
 
<!-- 638225395594806039--14166576050113124833 -->
 
 638225395594806039--14166576050113124833 
 
 1 out 
 2 p 
 
<!-- 15955734043749553171 -->
 
 15955734043749553171 
 
<!-- 15955734043749553171--3368846892928075864 -->
 
 15955734043749553171--3368846892928075864 
 
 1 out 
 1 out